# Multi Layer Perceptron

- Review : XOR
- Multi Layer Perceptron
- Backpropagation
- Code : xor-nn
- Code : xor-nn-wide-deep

### XOR

![](https://projectiot123.com/wp-content/uploads/2019/05/XOR-GATE.jpg)

이문제는 이전 09에서도 봤듯이 Single Layer로는 해결이 불가능 하다.  
고로 Multi Layer가 필요한데...  

- We need to use MLP(Multi Layer Perceptorn) 
- No one on earth had found a viable way to train MLPs good enough to learn such simple functions
(by Marvin Minsky,)

위 같이 말했지만 __Backpropagation__을 이용한다면??

### Backpropagation

http://aikorea.org/cs231n/optimization-2/  
Backpropagation에 대해서만 다루는 글은 제 깃헙에 추후 올리도록 하겠습니다. 

### Backpropagtaion with Pytorch

In [2]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
## 이 코드의 문제점은 !!??????????

X = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
Y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

## nn Layers
w1 = torch.FloatTensor(2,2).to(device)
b1 = torch.FloatTensor(2).to(device)
w2 = torch.FloatTensor(2,1).to(device)
b2 = torch.FloatTensor(1).to(device)
learning_rate = 0.1

def sigmoid(x) :
    # sigmoid function
    return 1.0 / (1.0 + torch.exp(-x))

def sigmoid_prime(x) : 
    # derivative of the sigmoid function (도함수)
    return sigmoid(x) * (1 - sigmoid(x)) 

for step in range(10001) :
    # forward
    l1 = torch.add(torch.matmul(X, w1), b1)
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1, w2), b2)
    Y_pred = sigmoid(l2)
    
    # cost를 구하기 위해 Binary Cross Entropy Loss를 사용하기로 합니다. 
    cost = - torch.mean(Y * torch.log(Y_pred) + (1 - Y) * torch.log(1 - Y_pred))
    
    
    # Back prop (chain rule)
    # Loss derivative
    
    # Binary cross Entropy Loss에 대한 미분 
    d_Y_pred = (Y_pred -Y) / (Y_pred * (1.0 - Y_pred) + 1e-7)
    # 1e-7는 0이 나눠지는것을 방지하기위함
    
    # Layer 2
    d_l2 = d_Y_pred * sigmoid_prime(l2)
    d_b2 = d_l2
    d_w2 = torch.matmul(torch.transpose(a1,0,1),d_b2)
    # transpose 는 차원축을 정해서 Transpose 가능함
    # (10,5) 를 torch.transpose(a1,0,1) 를 하면 (5,10)이된다.
    
    
    # Layer 1
    d_a1 = torch.matmul(d_b2, torch.transpose(w2,0,1))
    d_l1 = d_a1 * sigmoid_prime(l1)
    d_b1 = d_l1
    d_w1 = torch.matmul(torch.transpose(X,0,1),d_b1)
    
    # Weight update 
    # Gradient descent minimize
    w1 = w1 - learning_rate * d_w1
    b1 = b1 - learning_rate * torch.mean(d_b1,0)
    w2 = w2 - learning_rate * d_w2
    b2 - b2 - learning_rate * torch.mean(d_b2,0)
    
    if step %100 == 0:
        print(step,cost.item())

0 0.6931471824645996
100 0.6931471824645996
200 0.6931471824645996
300 0.6931471824645996
400 0.6931471824645996
500 0.6931471824645996
600 0.6931471824645996


왠지 모르게 안된다... 그냥 하던데로 하보고 다시 코드를 보자

#### 09 에서 봤던 코드 와 매우 유사하다 하지만 linear2rk 추가된것을 볼 수 있다.

In [5]:
X = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
Y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

# nn Layer
linear1 = torch.nn.Linear(2,2, bias=True)
linear2 = torch.nn.Linear(2,1, bias=True) 
sigmoid = torch.nn.Sigmoid()
model = torch.nn.Sequential(linear1, sigmoid , linear2, sigmoid).to(device)

# define cost/loss $ optimizer
criterion = torch.nn.BCELoss().to(device) # X값이 0,1과 1을 분리하는 Binary Classification Task 이므로 Loss로는 Binary Cross Entropy Loss를 사용합니다.
optimizer = torch.optim.SGD(model.parameters(), lr=1)
# Optimizer로는 이전에도 이용하는 statistic gradient descent 를 사용

for step in range(1001) :
    optimizer.zero_grad()
    hypothesis = model(X)
    
    # cost/loss function
    cost = criterion(hypothesis, Y)
    # model이 X,Y 게이트와 같은 값이 나올수 있도록 학습
    cost.backward()
    optimizer.step()
    if step % 100 == 0 :
        print(step, cost.item())

0 0.8579397797584534
100 0.6918308734893799
200 0.6874244213104248
300 0.6484621167182922
400 0.5045989751815796
500 0.4177666902542114
600 0.3875471353530884
700 0.3743336796760559
800 0.3672516345977783
900 0.362920880317688
1000 0.3600276708602905


##### 성공!!!

응용해서 3개 4개의 Multi Layer또한 구성 가능하다